## Turkish Instructions Retrieval Embedding

In [ ]:
!pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 18.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.8 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


#### Imports, Set Path and Constant Values

In [1]:
import sys
import os

# Set Constants
# If use Google Colab
use_colab = True
if use_colab:
    from google.colab import drive
    drive.mount('/content/drive')
    ROOT_PATH = "/content/drive/MyDrive/Colab Notebooks/HW3"  # Google Drive için kök dizin
    sys.path.append(os.path.join(ROOT_PATH, 'src'))
    device = "cuda"
else:
    #ROOT_PATH = os.getcwd()
    device = "cpu"

CACHE_PATH = os.path.join(ROOT_PATH, 'cache')
DATA_PATH = os.path.join(ROOT_PATH, 'data')
RAW_DATA_PATH = os.path.join(DATA_PATH, 'raw')
EMBEDDING_DATA_PATH = os.path.join(DATA_PATH, 'embedding')

# Create Folders
os.makedirs(RAW_DATA_PATH, exist_ok=True)
os.makedirs(EMBEDDING_DATA_PATH, exist_ok=True)
os.makedirs(CACHE_PATH, exist_ok=True)

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import warnings
from datetime import datetime

#from datasets import load_dataset
from utils import missing_values

In [4]:
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', None)

warnings.filterwarnings('ignore')

### Load Dataset: Turkish Instruction

In [ ]:
ds = load_dataset("merve/turkish_instructions")

README.md:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

instructions.csv:   0%|          | 0.00/21.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/51563 [00:00<?, ? examples/s]

In [ ]:
df_ins_data_all = ds['train'].to_pandas()
df_ins_data_all.head()

,Unnamed: 0,talimat,giriş,çıktı
0,0,Aşağıdaki bağlamda orijinal bir şey söyleyin: Bir iş görüşmesini yeni bitirdiniz ve işverene bu pozisyona olan ilginizi göstermek istiyorsunuz.,None,Ekibe katkıda bulunmaya başlamak ve bu alandaki becerilerimi daha da geliştirmek için çok istekliyim.
1,1,"Aşağıdaki makale için bir başlık önerin: Bu makale, işletmelerin müşterilerle iletişim kurma biçiminde devrim yaratmada internetin oynadığı rolü incelemektedir.",None,"""Dijital İletişimin Gücü: İnternet İş Ortamını Nasıl Dönüştürüyor"""
2,2,Şu konuyla ilgili 5 anahtar kelimeden oluşan bir liste oluşturun: iklim değişikliği,None,1. Karbon emisyonları 2. Yenilenebilir enerji 3. Küresel ısınma 4. İklim eylemi 5. Sürdürülebilir kalkınma
3,3,Bu cümleyi pasiften aktife değiştirin: Yeni politika belediye başkanı tarafından açıklandı.,None,Belediye başkanı yeni politikayı açıkladı.
4,4,Aşağıdaki fenomeni tanımlayan üç bilimsel terim tanımlayın: İki nesneyi birbirine çeken gizemli bir güç.,None,"Yerçekimi kuvveti, Newton'un evrensel çekim yasası ve elektrostatik kuvvet."


In [ ]:
# 'Unnamed: 0' sütununu kaldır
if 'Unnamed: 0' in df_ins_data_all.columns:
    df_ins_data_all = df_ins_data_all.drop(columns=['Unnamed: 0'])

In [ ]:
df_ins_data_all.columns

Index(['talimat', ' giriş', ' çıktı'], dtype='object')

In [ ]:
# df_ins_data_all.columns[1] rename to "giris" and [2] rename to "cikis"
df_ins_data_all.rename(columns={df_ins_data_all.columns[0]: 'talimat'}, inplace=True)
df_ins_data_all.rename(columns={df_ins_data_all.columns[1]: 'giris'}, inplace=True)
df_ins_data_all.rename(columns={df_ins_data_all.columns[2]: 'cevap'}, inplace=True)

In [ ]:
df_ins_data_all.to_csv(os.path.join(RAW_DATA_PATH, "turkish_instructions.csv"), index=False, encoding='utf-8')
df_ins_data_all.head()

,talimat,giris,cevap
0,Aşağıdaki bağlamda orijinal bir şey söyleyin: Bir iş görüşmesini yeni bitirdiniz ve işverene bu pozisyona olan ilginizi göstermek istiyorsunuz.,None,Ekibe katkıda bulunmaya başlamak ve bu alandaki becerilerimi daha da geliştirmek için çok istekliyim.
1,"Aşağıdaki makale için bir başlık önerin: Bu makale, işletmelerin müşterilerle iletişim kurma biçiminde devrim yaratmada internetin oynadığı rolü incelemektedir.",None,"""Dijital İletişimin Gücü: İnternet İş Ortamını Nasıl Dönüştürüyor"""
2,Şu konuyla ilgili 5 anahtar kelimeden oluşan bir liste oluşturun: iklim değişikliği,None,1. Karbon emisyonları 2. Yenilenebilir enerji 3. Küresel ısınma 4. İklim eylemi 5. Sürdürülebilir kalkınma
3,Bu cümleyi pasiften aktife değiştirin: Yeni politika belediye başkanı tarafından açıklandı.,None,Belediye başkanı yeni politikayı açıkladı.
4,Aşağıdaki fenomeni tanımlayan üç bilimsel terim tanımlayın: İki nesneyi birbirine çeken gizemli bir güç.,None,"Yerçekimi kuvveti, Newton'un evrensel çekim yasası ve elektrostatik kuvvet."


### Show Metadata, Shape, Statistics and Missing Table

In [ ]:
df_ins_data_all.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51563 entries, 0 to 51562
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   talimat  51563 non-null  object
 1   giris    19088 non-null  object
 2   cevap    51563 non-null  object
dtypes: object(3)
memory usage: 1.2+ MB


In [ ]:
df_ins_data_all.shape

(51563, 3)

In [ ]:
missing_values(df=df_ins_data_all, threshold=0, asc_sorting=False)

,Count,Percent
Columns,,
giris,32475,62.98121
talimat,0,0.00000
cevap,0,0.00000


In [ ]:
col_list = df_ins_data_all.columns.to_list()

for col in col_list:
    df_ins_data_all[f"{col}Length"] = df_ins_data_all[col].apply(lambda x: len(str(x)) if pd.notnull(x) else 0)
    df_ins_data_all[f"{col}WordCount"] = df_ins_data_all[col].apply(lambda x: len(str(x).split()) if pd.notnull(x) else 0)

In [ ]:
df_ins_data_all.head()

,talimat,giris,cevap,talimatLength,talimatWordCount,girisLength,girisWordCount,cevapLength,cevapWordCount
0,Aşağıdaki bağlamda orijinal bir şey söyleyin: Bir iş görüşmesini yeni bitirdiniz ve işverene bu pozisyona olan ilginizi göstermek istiyorsunuz.,None,Ekibe katkıda bulunmaya başlamak ve bu alandaki becerilerimi daha da geliştirmek için çok istekliyim.,144,19,0,0,102,14
1,"Aşağıdaki makale için bir başlık önerin: Bu makale, işletmelerin müşterilerle iletişim kurma biçiminde devrim yaratmada internetin oynadığı rolü incelemektedir.",None,"""Dijital İletişimin Gücü: İnternet İş Ortamını Nasıl Dönüştürüyor""",161,19,0,0,67,8
2,Şu konuyla ilgili 5 anahtar kelimeden oluşan bir liste oluşturun: iklim değişikliği,None,1. Karbon emisyonları 2. Yenilenebilir enerji 3. Küresel ısınma 4. İklim eylemi 5. Sürdürülebilir kalkınma,84,12,0,0,107,15
3,Bu cümleyi pasiften aktife değiştirin: Yeni politika belediye başkanı tarafından açıklandı.,None,Belediye başkanı yeni politikayı açıkladı.,92,11,0,0,43,5
4,Aşağıdaki fenomeni tanımlayan üç bilimsel terim tanımlayın: İki nesneyi birbirine çeken gizemli bir güç.,None,"Yerçekimi kuvveti, Newton'un evrensel çekim yasası ve elektrostatik kuvvet.",104,14,0,0,76,9


In [ ]:
df_ins_data_all[["talimatLength", "girisLength", "cevapLength", "talimatWordCount", "girisWordCount", "cevapWordCount"]].describe(percentiles=[0.80, 0.85, 0.90, 0.95, 0.99]).T

,count,mean,std,min,50%,80%,85%,90%,95%,99%,max
talimatLength,51563.0,63.265403,26.986467,10.0,59.0,78.0,83.0,90.0,103.0,145.0,1056.0
girisLength,51563.0,23.468126,71.065212,0.0,0.0,34.0,45.0,62.0,103.0,289.0,2653.0
cevapLength,51563.0,279.215523,281.753389,1.0,196.0,486.0,548.7,628.0,775.0,1241.0,3607.0
talimatWordCount,51563.0,8.396563,3.836307,2.0,8.0,10.0,11.0,12.0,14.0,20.0,145.0
girisWordCount,51563.0,3.243392,9.317139,0.0,0.0,5.0,6.0,9.0,14.0,38.0,353.0
cevapWordCount,51563.0,37.181409,37.274826,1.0,27.0,64.0,72.0,82.0,102.0,166.0,495.0


### Split Data

In [ ]:
df_ins_data_all['soru'] = df_ins_data_all['talimat'].fillna('') + ' ' + df_ins_data_all['giris'].fillna('')
df_ins_data_all[["talimat", "giris", "soru", "cevap"]].head()

,talimat,giris,soru,cevap
0,Aşağıdaki bağlamda orijinal bir şey söyleyin: Bir iş görüşmesini yeni bitirdiniz ve işverene bu pozisyona olan ilginizi göstermek istiyorsunuz.,None,Aşağıdaki bağlamda orijinal bir şey söyleyin: Bir iş görüşmesini yeni bitirdiniz ve işverene bu pozisyona olan ilginizi göstermek istiyorsunuz.,Ekibe katkıda bulunmaya başlamak ve bu alandaki becerilerimi daha da geliştirmek için çok istekliyim.
1,"Aşağıdaki makale için bir başlık önerin: Bu makale, işletmelerin müşterilerle iletişim kurma biçiminde devrim yaratmada internetin oynadığı rolü incelemektedir.",None,"Aşağıdaki makale için bir başlık önerin: Bu makale, işletmelerin müşterilerle iletişim kurma biçiminde devrim yaratmada internetin oynadığı rolü incelemektedir.","""Dijital İletişimin Gücü: İnternet İş Ortamını Nasıl Dönüştürüyor"""
2,Şu konuyla ilgili 5 anahtar kelimeden oluşan bir liste oluşturun: iklim değişikliği,None,Şu konuyla ilgili 5 anahtar kelimeden oluşan bir liste oluşturun: iklim değişikliği,1. Karbon emisyonları 2. Yenilenebilir enerji 3. Küresel ısınma 4. İklim eylemi 5. Sürdürülebilir kalkınma
3,Bu cümleyi pasiften aktife değiştirin: Yeni politika belediye başkanı tarafından açıklandı.,None,Bu cümleyi pasiften aktife değiştirin: Yeni politika belediye başkanı tarafından açıklandı.,Belediye başkanı yeni politikayı açıkladı.
4,Aşağıdaki fenomeni tanımlayan üç bilimsel terim tanımlayın: İki nesneyi birbirine çeken gizemli bir güç.,None,Aşağıdaki fenomeni tanımlayan üç bilimsel terim tanımlayın: İki nesneyi birbirine çeken gizemli bir güç.,"Yerçekimi kuvveti, Newton'un evrensel çekim yasası ve elektrostatik kuvvet."


In [ ]:
df_sample_1 = df_ins_data_all[df_ins_data_all['giris'].notna()].sample(n=1000, random_state=42)
df_sample_1_idx = df_sample_1.index
df_sample_1.head()

df_sample_2 = df_ins_data_all[~(df_ins_data_all['giris'].notna())].sample(n=1000, random_state=42)
df_sample_2_idx = df_sample_2.index
df_sample_2.head()

df_sample = pd.concat([df_sample_1, df_sample_2])
df_sample['soru'] = df_sample['talimat'].fillna('') + " " + df_sample['giris'].fillna('')

df_ins_data_all[col].apply(lambda x: len(str(x).split()) if pd.notnull(x) else 0)

df_sample.to_csv(os.path.join(RAW_DATA_PATH, "sampled_turkish_instructions.csv"), index=False)

df_sample = df_sample[["talimat", "giris", "soru", "cevap"]].reset_index(drop=True)
df_sample = pd.read_csv(os.path.join(RAW_DATA_PATH, "sampled_turkish_instructions.csv"))
df_sample.head()

,talimat,giris,cevap,talimatLength,talimatWordCount,girisLength,girisWordCount,cevapLength,cevapWordCount,soru
0,Belirli bir konu hakkında bir makale yazın.,Konu: Zaman Yönetimi,"Zaman yönetimi, hayatta başarılı olmak için sahip olunması gereken önemli bir beceridir. Gününüzden en iyi şekilde yararlanabilmeniz için görevlere öncelik vermenizi ve zamanınızı en iyi şekilde değerlendirmenizi sağlar. Etkili zaman yönetimine sahip olmak aynı zamanda odaklanmanıza ve doğru yolda kalmanıza yardımcı olur ve ertelemeyi önler. Zamanınızı etkili bir şekilde yönetmenin bir yolu, bir program oluşturmak ve ona bağlı kalmaktır. Günün belirli saatlerini belirli etkinliklere ayırmanız ve bu saatlere saygı duymanız önemlidir. Örneğin spor yapmak istiyorsanız her gün aynı saatte bir saat ayırın ve o randevuya mutlaka uyun. Görevleriniz için ulaşılabilir hedefler ve zaman çizelgesi belirlediğinizden de emin olmalısınız. Bunu yaparak, daha büyük görevleri yönetilebilir parçalara ayırabilir ve odaklanmanızı ve motive olmanızı sağlayacak somut hedeflere ve teslim tarihlerine sahip olabilirsiniz. Zaman yönetimini iyileştirmenin bir başka yolu da düzenli kalmak ve bir ""yapılacaklar"" listesi oluşturmaktır. Yapılacaklar listesi, önemli görevleri takip etmenize ve en önemlilerine öncelik vermenize yardımcı olacaktır. Ayrıca, görevleri unutma olasılığını azaltır ve tamamlamanız gereken görevleri hatırlatır. Son olarak, dikkat dağıtıcı unsurları yönetmek, etkili zaman yönetimi için çok önemlidir. Verimli kalmak için, gecikmelere neden olabileceğinden ve üretkenliği engelleyebileceğinden, dikkat dağıtıcı unsurları sınırlamak önemlidir. Düzenli kalmak ve mola vermek bunu önlemeye yardımcı olabilir. Genel olarak, zaman yönetimi hayatta başarılı olmak için sahip olunması gereken çok önemli bir beceridir. Programlar oluşturarak, ulaşılabilir hedefler belirleyerek, düzenli kalarak ve dikkat dağıtıcı unsurları sınırlayarak, kişi zamanını etkili bir şekilde yönetebilir ve daha fazlasını başarabilir.",44,7,20,3,1817,228,Belirli bir konu hakkında bir makale yazın. Konu: Zaman Yönetimi
1,Aynı konudaki iki makaleyi alaka düzeyine ve doğruluğuna göre karşılaştırın.,"Madde 1: ""Büyük Bankalar İklim Değişikliğini Önemsiyor"" Madde 2: ""BP'nin İklim Değişikliğiyle Mücadele Planı""","Birinci madde, birçok büyük bankanın çabalarını tartışarak, bankacılık sektörünün iklim değişikliğine yaklaşımına daha geniş bir genel bakış sunuyor. Büyük bankaların motivasyonlarını ve girişimlerini ve potansiyel etkilerini derinlemesine inceler. İkinci madde, BP'nin belirli girişimlerine odaklanmaktadır ve şirketin iklim değişikliği planları hakkındaki bilgilerde daha ayrıntılı olabilir. Her iki makale de konuyla alakalı, ancak birinci makale bankacılık sektörünün iklim değişikliğine tepkisine daha kapsamlı bir bakış sunuyor.",77,10,110,14,534,63,"Aynı konudaki iki makaleyi alaka düzeyine ve doğruluğuna göre karşılaştırın. Madde 1: ""Büyük Bankalar İklim Değişikliğini Önemsiyor"" Madde 2: ""BP'nin İklim Değişikliğiyle Mücadele Planı"""
2,Sağlanan kelimeleri kullanarak çılgın bir kütüphane oluşturun.,"at, hindi, yelkenli",At macera yaşamak istediğinde bir hindiye atlar ve bir yelkenliyle yola çıkar.,63,7,20,3,79,12,"Sağlanan kelimeleri kullanarak çılgın bir kütüphane oluşturun. at, hindi, yelkenli"
3,"Bir senaryo verildiğinde, kullanıcının bir sonraki eyleminin ne olacağını tahmin edin",John yeni bir telefon aldı ve ekran kilidini ayarlıyor,"John muhtemelen bir ekran kilidi türü (örn. parola, PIN, parmak izi vb.) seçecek ve bunu ayarlamak için gereken bilgileri girecektir.",86,11,55,9,133,20,"Bir senaryo verildiğinde, kullanıcının bir sonraki eyleminin ne olacağını tahmin edin John yeni bir telefon aldı ve ekran kilidini ayarlıyor"
4,Aşağıdaki durumdaki kişi hakkında güzel bir şey söyleyin,Yanan bir binadan birini kurtaran bir itfaiyeci,"İtfaiyeci, başka birini kurtarmak için kendi güvenliğini riske attığı için kahraman ve özverilidir.",57,8,48,7,100,13,Aşağıdaki durumdaki kişi hakkında gü

In [ ]:
df_sample.sample(5)

,talimat,giris,cevap,talimatLength,talimatWordCount,girisLength,girisWordCount,cevapLength,cevapWordCount,soru
779,"Bir film adı verildiğinde, onu klasik yapan üç şeyi listeleyin.",vaftiz babası,"Godfather, Marlon Brando, Al Pacino ve Robert Duvall'ın ikonik performansları, unutulmaz müzikleri ve güçlü bir mafya organizasyonundaki karmaşık bir aile dinamiğini takip eden karmaşık olay örgüsü nedeniyle bir klasik olarak kabul ediliyor. Film, şimdiye kadar yapılmış en etkili filmlerden biri olarak kabul ediliyor ve etkisi bugün hala eğlence endüstrisinde geniş çapta hissediliyor.",64,10,14,2,387,52,"Bir film adı verildiğinde, onu klasik yapan üç şeyi listeleyin. vaftiz babası"
1355,Evrim ve doğal seçilim arasındaki üç farkı söyleyin.,NaN,"Evrim, zaman içinde organizma popülasyonlarındaki değişim sürecidir, doğal seçilim ise bir popülasyonda belirli özelliklerin az ya da çok yaygın hale geldiği süreçtir. Evrim uzun sürelerde gerçekleşirken, doğal seçilim daha kısa sürelerde gerçekleşir. Evrim bir teori, doğal seçilim ise bir mekanizmadır.",53,8,0,0,305,40,Evrim ve doğal seçilim arasındaki üç farkı söyleyin.
1362,Bir dil ailesinin en iyi örneği nedir?,NaN,"Hint-Avrupa dil ailesi, bir dil ailesinin en iyi örneğidir. Avrupa, Kuzey ve Güney Amerika, Hint-İran ve Hindistan-Pakistan alt kıtasında konuşulan dilleri içerir. Hint-Avrupa dil ailesindeki diller arasında İngilizce, İspanyolca, Fransızca, Almanca, Felemenkçe, Portekizce, Rusça, Yunanca ve Sanskritçe bulunur.",39,7,0,0,312,38,Bir dil ailesinin en iyi örneği nedir?
345,Aşağıdaki ifadenin ahlaki sonuçlarını analiz edin.,"""Amaçlar araçları haklı çıkarır""","""Sonuçlar araçları haklı çıkarır"" ifadesi, bir hedefe ulaşmak için yapılan herhangi bir eylemin, sonuçları ne olursa olsun ahlaki olarak kabul edilebilir olduğunu öne sürer. Bu, eylemin sonucu süreçten daha önemli görüldüğü için etik olmayan ve potansiyel olarak tehlikeli kararlara yol açabilir. Ayrıca, her insanın saygı ve nezaketle muamele görme hakkı gibi ahlaki ve etik ilkelerle de çatışabilir. Bireyler, bu sonuçlara nasıl ulaşıldığına değil, yalnızca nihai sonuçlara bakarsa, o zaman kararları başkalarının zararına olabilir.",51,6,33,4,534,73,"Aşağıdaki ifadenin ahlaki sonuçlarını analiz edin. ""Amaçlar araçları haklı çıkarır"""
1536,Doğa ile ilgili kısa bir şiir yazınız.,NaN,"Güzel gökyüzü bir mavi senfonisi, Doğanın harikaları her zaman göz önünde. Ağaçlar ve çiçekler, havayı nasıl dolduruyorlar, Paylaşmamız için gezegenimizin harikası. Hafif bir esinti, yeniden hayat veren, Çünkü doğada ne cesur ve doğru bir manzara.",39,7,0,0,248,35,Doğa ile ilgili kısa bir şiir yazınız.


In [ ]:
df_sample.shape

(2000, 10)

In [10]:
def load_or_download_model(model_name, device):
    model_path = os.path.join(CACHE_PATH, model_name)
    if os.path.exists(model_path):
        print(f"Loading Cached Model: {model_name}")
        return SentenceTransformer(model_path, device=device, trust_remote_code=True)
    else:
        print(f"Downloading Model: {model_name}")
        model = SentenceTransformer(model_name, device=device, trust_remote_code=True)
        model.save(model_path)
        return model

In [ ]:
from sentence_transformers import SentenceTransformer

embedding_models = [
    'sentence-transformers/all-MiniLM-L12-v2',
    "jinaai/jina-embeddings-v3",
    "intfloat/multilingual-e5-large-instruct",
    "BAAI/bge-m3",
    "ytu-ce-cosmos/turkish-colbert",
    "thenlper/gte-large"
]

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
for idx, model_name in enumerate(embedding_models):
    try:
        embedding_start_time = datetime.now()
        print(f"{idx+1}-Embedding Model Name: {model_name} - {embedding_start_time}")
        model = load_or_download_model(model_name, device)

        # Sorular için embedding
        print("Sorular için embedding hesaplanıyor...")
        question_embeddings = model.encode(df_sample['soru'].tolist(), show_progress_bar=True)
        question_embedding_dim = len(question_embeddings[0])
        print(f"Embedding Dimension: {question_embedding_dim}")
        question_columns = [f'question_embedding_{i}' for i in range(question_embedding_dim)]
        question_df = pd.DataFrame(question_embeddings, columns=question_columns)
        question_df = pd.concat([df_sample[['soru']], question_df], axis=1)
        question_output_file = os.path.join(
          EMBEDDING_DATA_PATH, f"question_{model_name.replace('/', '__')}_embeddings.csv.gz"
        )
        question_df.to_csv(question_output_file, compression='gzip', index=False)

        # Cevaplar için embedding
        print("Cevaplar için embedding hesaplanıyor...")
        answer_embeddings = model.encode(df_sample['cevap'].tolist(), show_progress_bar=True)
        answer_embedding_dim = len(answer_embeddings[0])
        print(f"Embedding Dimension: {answer_embedding_dim}")
        answer_columns = [f'answer_embedding_{i}' for i in range(answer_embedding_dim)]
        answer_df = pd.DataFrame(answer_embeddings, columns=answer_columns)
        answer_df = pd.concat([df_sample[['cevap']], answer_df], axis=1)
        answer_output_file = os.path.join(
          EMBEDDING_DATA_PATH, f"answer_{model_name.replace('/', '__')}_embeddings.csv.gz"
        )
        answer_df.to_csv(answer_output_file, compression='gzip', index=False)

        print(f"{model_name} için embedding işlemi tamamlandı. Embedding Duration: {datetime.now() - embedding_start_time}")
        print("+ " * 15)
    except Exception as exc:
        print(f"{model_name} için hata oluştu: {exc}")
        print("x " * 15)

1-Embedding Model Name: sentence-transformers/all-MiniLM-L12-v2 - 2024-12-17 18:30:15.483732
Sorular için embedding hesaplanıyor...


Batches:   0%|          | 0/63 [00:00<?, ?it/s]

Embedding Dimension: 384
Cevaplar için embedding hesaplanıyor...


Batches:   0%|          | 0/63 [00:00<?, ?it/s]

Embedding Dimension: 384
sentence-transformers/all-MiniLM-L12-v2 için embedding işlemi tamamlandı. Embedding Duration: 0:00:12.803942
+ + + + + + + + + + + + + + + 
2-Embedding Model Name: jinaai/jina-embeddings-v3 - 2024-12-17 18:30:28.287796


...

Sorular için embedding hesaplanıyor...


Batches:   0%|          | 0/63 [00:00<?, ?it/s]

Embedding Dimension: 1024
Cevaplar için embedding hesaplanıyor...


Batches:   0%|          | 0/63 [00:00<?, ?it/s]

Embedding Dimension: 1024
thenlper/gte-large için embedding işlemi tamamlandı. Embedding Duration: 0:01:06.590133
+ + + + + + + + + + + + + + + 


### Embedding

In [ ]:
# df_emb_data_que_1 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "question_sentence-transformers__all-MiniLM-L12-v2_embeddings.csv.gz"), compression='gzip')
# df_emb_data_ans_1 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "answer_sentence-transformers__all-MiniLM-L12-v2_embeddings.csv.gz"), compression='gzip')

# df_emb_data_que_2 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "question_jinaai__jina-embeddings-v3_embeddings.csv.gz"), compression='gzip')
# df_emb_data_ans_2 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "answer_jinaai__jina-embeddings-v3_embeddings.csv.gz"), compression='gzip')

# df_emb_data_que_3 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "question_intfloat__multilingual-e5-large-instruct_embeddings.csv.gz"), compression='gzip')
# df_emb_data_ans_3 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "answer_intfloat__multilingual-e5-large-instruct_embeddings.csv.gz"), compression='gzip')

# df_emb_data_que_4 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "question_BAAI__bge-m3_embeddings.csv.gz"), compression='gzip')
# df_emb_data_ans_4 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "answer_BAAI__bge-m3_embeddings.csv.gz"), compression='gzip')

# df_emb_data_que_5 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "question_ytu-ce-cosmos__turkish-colbert_embeddings.csv.gz"), compression='gzip')
# df_emb_data_ans_5 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "answer_ytu-ce-cosmos__turkish-colbert_embeddings.csv.gz"), compression='gzip')

# df_emb_data_que_6 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "question_thenlper__gte-large_embeddings.csv.gz"), compression='gzip')
# df_emb_data_ans_6 = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, "answer_thenlper__gte-large_embeddings.csv.gz"), compression='gzip')

In [ ]:
for idx, f in enumerate(os.listdir(EMBEDDING_DATA_PATH)):
    # print(f)
    # if f.split("_")[0] == "answer":
    if f.split("_")[0] in ["answer", "question"]:
        df_emb_data = pd.read_csv(os.path.join(EMBEDDING_DATA_PATH, f), compression='gzip')
        print(f"{idx+1} - Emb. File Name:{f} & Shape: {df_emb_data.shape}")
        print(df_emb_data.head(0))
        # print(f"Columns:{df_emb_data.columns.to_list()}")
        print("+ " * 25)

1 - Emb. File Name:question_sentence-transformers__all-MiniLM-L12-v2_embeddings.csv.gz & Shape: (2000, 385)
Empty DataFrame
Columns: [soru, question_embedding_0, question_embedding_1, question_embedding_2, question_embedding_3, question_embedding_4, question_embedding_5, question_embedding_6, question_embedding_7, question_embedding_8, question_embedding_9, question_embedding_10, question_embedding_11, question_embedding_12, question_embedding_13, question_embedding_14, question_embedding_15, question_embedding_16, question_embedding_17, question_embedding_18, question_embedding_19, question_embedding_20, question_embedding_21, question_embedding_22, question_embedding_23, question_embedding_24, question_embedding_25, question_embedding_26, question_embedding_27, question_embedding_28, question_embedding_29, question_embedding_30, question_embedding_31, question_embedding_32, question_embedding_33, question_embedding_34, question_embedding_35, question_embedding_36, question_embedding_

In [5]:
df_sample = pd.read_csv(os.path.join(RAW_DATA_PATH, "sampled_turkish_instructions.csv"))


In [6]:
df_sample.head()

,talimat,giris,cevap,talimatLength,talimatWordCount,girisLength,girisWordCount,cevapLength,cevapWordCount,soru
0,Belirli bir konu hakkında bir makale yazın.,Konu: Zaman Yönetimi,"Zaman yönetimi, hayatta başarılı olmak için sahip olunması gereken önemli bir beceridir. Gününüzden en iyi şekilde yararlanabilmeniz için görevlere öncelik vermenizi ve zamanınızı en iyi şekilde değerlendirmenizi sağlar. Etkili zaman yönetimine sahip olmak aynı zamanda odaklanmanıza ve doğru yolda kalmanıza yardımcı olur ve ertelemeyi önler. Zamanınızı etkili bir şekilde yönetmenin bir yolu, bir program oluşturmak ve ona bağlı kalmaktır. Günün belirli saatlerini belirli etkinliklere ayırmanız ve bu saatlere saygı duymanız önemlidir. Örneğin spor yapmak istiyorsanız her gün aynı saatte bir saat ayırın ve o randevuya mutlaka uyun. Görevleriniz için ulaşılabilir hedefler ve zaman çizelgesi belirlediğinizden de emin olmalısınız. Bunu yaparak, daha büyük görevleri yönetilebilir parçalara ayırabilir ve odaklanmanızı ve motive olmanızı sağlayacak somut hedeflere ve teslim tarihlerine sahip olabilirsiniz. Zaman yönetimini iyileştirmenin bir başka yolu da düzenli kalmak ve bir ""yapılacaklar"" listesi oluşturmaktır. Yapılacaklar listesi, önemli görevleri takip etmenize ve en önemlilerine öncelik vermenize yardımcı olacaktır. Ayrıca, görevleri unutma olasılığını azaltır ve tamamlamanız gereken görevleri hatırlatır. Son olarak, dikkat dağıtıcı unsurları yönetmek, etkili zaman yönetimi için çok önemlidir. Verimli kalmak için, gecikmelere neden olabileceğinden ve üretkenliği engelleyebileceğinden, dikkat dağıtıcı unsurları sınırlamak önemlidir. Düzenli kalmak ve mola vermek bunu önlemeye yardımcı olabilir. Genel olarak, zaman yönetimi hayatta başarılı olmak için sahip olunması gereken çok önemli bir beceridir. Programlar oluşturarak, ulaşılabilir hedefler belirleyerek, düzenli kalarak ve dikkat dağıtıcı unsurları sınırlayarak, kişi zamanını etkili bir şekilde yönetebilir ve daha fazlasını başarabilir.",44,7,20,3,1817,228,Belirli bir konu hakkında bir makale yazın. Konu: Zaman Yönetimi
1,Aynı konudaki iki makaleyi alaka düzeyine ve doğruluğuna göre karşılaştırın.,"Madde 1: ""Büyük Bankalar İklim Değişikliğini Önemsiyor"" Madde 2: ""BP'nin İklim Değişikliğiyle Mücadele Planı""","Birinci madde, birçok büyük bankanın çabalarını tartışarak, bankacılık sektörünün iklim değişikliğine yaklaşımına daha geniş bir genel bakış sunuyor. Büyük bankaların motivasyonlarını ve girişimlerini ve potansiyel etkilerini derinlemesine inceler. İkinci madde, BP'nin belirli girişimlerine odaklanmaktadır ve şirketin iklim değişikliği planları hakkındaki bilgilerde daha ayrıntılı olabilir. Her iki makale de konuyla alakalı, ancak birinci makale bankacılık sektörünün iklim değişikliğine tepkisine daha kapsamlı bir bakış sunuyor.",77,10,110,14,534,63,"Aynı konudaki iki makaleyi alaka düzeyine ve doğruluğuna göre karşılaştırın. Madde 1: ""Büyük Bankalar İklim Değişikliğini Önemsiyor"" Madde 2: ""BP'nin İklim Değişikliğiyle Mücadele Planı"""
2,Sağlanan kelimeleri kullanarak çılgın bir kütüphane oluşturun.,"at, hindi, yelkenli",At macera yaşamak istediğinde bir hindiye atlar ve bir yelkenliyle yola çıkar.,63,7,20,3,79,12,"Sağlanan kelimeleri kullanarak çılgın bir kütüphane oluşturun. at, hindi, yelkenli"
3,"Bir senaryo verildiğinde, kullanıcının bir sonraki eyleminin ne olacağını tahmin edin",John yeni bir telefon aldı ve ekran kilidini ayarlıyor,"John muhtemelen bir ekran kilidi türü (örn. parola, PIN, parmak izi vb.) seçecek ve bunu ayarlamak için gereken bilgileri girecektir.",86,11,55,9,133,20,"Bir senaryo verildiğinde, kullanıcının bir sonraki eyleminin ne olacağını tahmin edin John yeni bir telefon aldı ve ekran kilidini ayarlıyor"
4,Aşağıdaki durumdaki kişi hakkında güzel bir şey söyleyin,Yanan bir binadan birini kurtaran bir itfaiyeci,"İtfaiyeci, başka birini kurtarmak için kendi güvenliğini riske attığı için kahraman ve özverilidir.",57,8,48,7,100,13,Aşağıdaki durumdaki kişi hakkında gü

In [13]:
from sentence_transformers import SentenceTransformer

embedding_models = [
    "ytu-ce-cosmos/turkish-colbert"
]

In [8]:
def preprocess_texts(texts, model_name):
    """
    Yalnızca ytu-ce-cosmos için özel bir dönüşüm uygulanır.
    Diğer modeller için metin olduğu gibi döndürülür.
    """
    if model_name == "ytu-ce-cosmos/turkish-colbert":
        return [text.replace("I", "ı").lower() for text in texts]
    return texts

In [14]:
for idx, model_name in enumerate(embedding_models):
    try:
        embedding_start_time = datetime.now()
        print(f"{idx+1}-Embedding Model Name: {model_name} - {embedding_start_time}")

        # Modeli yükle veya indir
        model = load_or_download_model(model_name, device)

        # Sorular için embedding
        print("Sorular için embedding hesaplanıyor...")
        preprocessed_questions = preprocess_texts(df_sample['soru'].tolist(), model_name)
        question_embeddings = model.encode(preprocessed_questions, show_progress_bar=True)
        question_embedding_dim = len(question_embeddings[0])
        print(f"Embedding Dimension: {question_embedding_dim}")
        question_columns = [f'question_embedding_{i}' for i in range(question_embedding_dim)]
        question_df = pd.DataFrame(question_embeddings, columns=question_columns)
        question_df = pd.concat([df_sample[['soru']], question_df], axis=1)
        question_output_file = os.path.join(
            EMBEDDING_DATA_PATH, f"question_{model_name.replace('/', '__')}_embeddingsLower.csv.gz"
        )
        question_df.to_csv(question_output_file, compression='gzip', index=False)

        # Cevaplar için embedding
        print("Cevaplar için embedding hesaplanıyor...")
        preprocessed_answers = preprocess_texts(df_sample['cevap'].tolist(), model_name)
        answer_embeddings = model.encode(preprocessed_answers, show_progress_bar=True)
        answer_embedding_dim = len(answer_embeddings[0])
        print(f"Embedding Dimension: {answer_embedding_dim}")
        answer_columns = [f'answer_embedding_{i}' for i in range(answer_embedding_dim)]
        answer_df = pd.DataFrame(answer_embeddings, columns=answer_columns)
        answer_df = pd.concat([df_sample[['cevap']], answer_df], axis=1)
        answer_output_file = os.path.join(
            EMBEDDING_DATA_PATH, f"answer_{model_name.replace('/', '__')}_embeddingsLower.csv.gz"
        )
        answer_df.to_csv(answer_output_file, compression='gzip', index=False)

        print(f"{model_name} için embedding işlemi tamamlandı. Embedding Duration: {datetime.now() - embedding_start_time}")
        print("+ " * 15)
    except Exception as exc:
        print(f"{model_name} için hata oluştu: {exc}")
        print("x " * 15)

1-Embedding Model Name: ytu-ce-cosmos/turkish-colbert - 2024-12-25 08:12:31.403945
Loading Cached Model: ytu-ce-cosmos/turkish-colbert
Sorular için embedding hesaplanıyor...


Batches:   0%|          | 0/63 [00:00<?, ?it/s]

Embedding Dimension: 768
Cevaplar için embedding hesaplanıyor...


Batches:   0%|          | 0/63 [00:00<?, ?it/s]

Embedding Dimension: 768
ytu-ce-cosmos/turkish-colbert için embedding işlemi tamamlandı. Embedding Duration: 0:00:40.734266
+ + + + + + + + + + + + + + + 
